In [1]:
import re
import nltk
from nltk.corpus import brown
from collections import Counter
from nltk.util import ngrams
from itertools import pairwise, product
from more_itertools import windowed
import math
import string
import random
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.util import ngrams

Download corpus

In [2]:
nltk.download('brown')

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\KYRIAKOS\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [3]:
sents = brown.sents(categories = 'editorial')

Preprocess text and tokenize

In [4]:
def preprocess(sents):
    # Preprocess: lowercase + keep only words that are letters, comma, or period
    clean = [
        [w.lower() for w in sent if re.fullmatch(r'[a-zA-Z]+|[,.]', w)]
        for sent in sents
    ]
    return clean

In [5]:
def filterLen(sents, lowThr, upperThr):
    return [s for s in sents if lowThr <= len(s) <= upperThr]

In [6]:
def filterFreq(sents, threshold):
    """
    Keep only tokens that appear >= min_freq times in the provided portion.
    Others are dropped.
    """
    # Flatten and count
    counts = Counter(w for sent in sents for w in sent)
    # Build vocabulary
    vocab = {w for w, c in counts.items() if c >= threshold}
    # Filter sentences
    filteredSents = [[w if w in vocab else '<UNK>' for w in sent] for sent in sents]
    return filteredSents, vocab


Split into subsets

In [7]:
def splitSubsets(data):
    random.seed(31)
    random.shuffle(data)
    
    trainSize = int(0.8 * len(data))
    valSize = int(0.1 * len(data))
    
    train = data[:trainSize]
    val = data[trainSize:trainSize + valSize]
    test = data[trainSize + valSize:]
    
    return train, val, test

Build the 3 models

In [8]:
class BigramLM:
    def __init__(self, sents, vocab, alpha):
        self.sents = sents
        self.vocab = vocab
        self.alpha = alpha
        self.uniCounter = Counter()
        self.biCounter = Counter()
        self.buildCounts()
    
    def buildCounts(self):
        """
        Takes tokenized sentences.
        Populates unigram and bigram counters with <START>/<END> padding.
        """
        for sent in self.sents:
            paddedSent = ['<START>'] + sent + ['<END>']
            self.uniCounter.update([gram for gram in ngrams(paddedSent, 1)])
            self.biCounter.update([gram for gram in ngrams(paddedSent, 2)])

    def prob(self, w1, w2):
        uniCount = self.uniCounter.get((w1,), 0)
        biCount = self.biCounter.get((w1, w2), 0)
        V = len(self.vocab)
        
        return (biCount + self.alpha) / (uniCount + self.alpha * V)

    def crossEntropyAndPerplexity(self, sents):
        sumLogProb = 0
        nBigrams = 0
        for sent in sents:
            paddedSent = ['<START>'] + sent + ['<END>']
            for w1, w2 in pairwise(paddedSent):
                sumLogProb += math.log2(self.prob(w1, w2))
                nBigrams += 1
        hc = -sumLogProb / nBigrams
        ppl = 2 ** hc
        
        return hc, ppl


In [9]:
class TrigramLM:
    def __init__(self, sents, vocab, alpha):
        self.sents = sents
        self.vocab = vocab
        self.alpha = alpha
        self.biCounter = Counter()
        self.triCounter = Counter()
        self.buildCounts()
    
    def buildCounts(self):
        """
        Takes tokenized sentences.
        Populates bigram and trigram counters with <START1>/<START2>/<END> padding.
        """
        for sent in self.sents:
            paddedSent = ['<START1>', '<START2>'] + sent + ['<END>']
            self.biCounter.update([gram for gram in ngrams(paddedSent, 2)])
            self.triCounter.update([gram for gram in ngrams(paddedSent, 3)])

    def prob(self, w1, w2, w3):
        biCount = self.biCounter.get((w1, w2), 0)
        triCount = self.triCounter.get((w1, w2, w3), 0)
        V = len(self.vocab)
        
        return (triCount + self.alpha) / (biCount + self.alpha * V)
    
    def crossEntropyAndPerplexity(self, sents):
        sumLogProb = 0
        nTrigrams = 0
        for sent in sents:
            paddedSent = ['<START1>', '<START2>'] + sent + ['<END>']
            for w1, w2, w3 in windowed(paddedSent, 3):
                sumLogProb += math.log2(self.prob(w1, w2, w3))
                nTrigrams += 1
        hc = -sumLogProb / nTrigrams
        ppl = 2 ** hc
        return hc, ppl


In [10]:
class InterpolatedLM:
    def __init__(self, sents, vocab, alphaB, alphaT, l):
        self.sents = sents
        self.bigram = BigramLM(sents, vocab, alphaB)
        self.trigram = TrigramLM(sents, vocab, alphaT)
        self.l = l
    
    def prob(self, w1, w2, w3):
        """
        Interpolated probability of w3 given w1,w2.
        """
        triProb = self.trigram.prob(w1, w2, w3)   # from trigram counts
        biProb = self.bigram.prob(w2, w3)         # from bigram counts
        return self.l * triProb + (1 - self.l) * biProb

    def crossEntropyAndPerplexity(self, sents):
        sumLogProb = 0
        ngramCount = 0
        
        for sent in sents:
            # Add two start tokens for trigrams
            sent = ['<START>', '<START>'] + sent + ['<END>']
            
            for w1, w2, w3 in windowed(sent, 3):
                trigramProb = self.trigram.prob(w1, w2, w3)
                bigramProb = self.bigram.prob(w2, w3)
                
                # Interpolation: λ * P(trigram) + (1-λ) * P(bigram)
                interpolatedProb = self.l * trigramProb + (1 - self.l) * bigramProb
                sumLogProb += math.log2(interpolatedProb)
                ngramCount += 1
        
        crossEntropy = -sumLogProb / ngramCount
        perplexity = math.pow(2, crossEntropy)
        return crossEntropy, perplexity


In [11]:
clean = preprocess(sents)

In [12]:
train, val, test = splitSubsets(clean)

In [13]:
fltrain = filterLen(train, 5, 20)

In [14]:
len(train)

2397

In [15]:
len(fltrain)

1164

In [16]:
fsents, vocab = filterFreq(fltrain, 10)

Evaluate individually (cross entropy) with fixed parameters

In [17]:
bgrm = BigramLM(fsents, vocab, 2)

In [18]:
bgrm.crossEntropyAndPerplexity(val)

(7.032173231620663, 130.88656718033667)

In [19]:
tgrm = TrigramLM(fsents, vocab, 3)

In [20]:
tgrm.crossEntropyAndPerplexity(val)

(7.3277080179523795, 160.64229975586332)

In [21]:
inter = InterpolatedLM(fsents, vocab, 2, 3, 0.2)

In [22]:
inter.crossEntropyAndPerplexity(val)

(6.996003193004343, 127.64588268142266)

Explore parameters

In [23]:
candAlpha = [0.5, 1, 2, 3, 5, 10, 50]
candThreshold = [5, 10, 15, 20, 30]
candLambda = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

candidates = [candAlpha, candThreshold]

In [24]:
def tune(model):
    optimalCE = math.inf
    for a, t in product(*candidates):
        fsents, vocab = filterFreq(fltrain, t)
        ngrm = model(fsents, vocab, a)
        ce, _ = ngrm.crossEntropyAndPerplexity(val)
        if(ce < optimalCE):
            optimalCE = ce
            optimalA = a
            optimalT = t

    return optimalA, optimalT, optimalCE

In [25]:
tune(BigramLM)

(50, 30, 5.576321052454329)

In [26]:
tune(TrigramLM)

(50, 30, 5.7246604337433356)

In [27]:
fsents, vocab = filterFreq(fltrain, 30)
optimalCE = math.inf
for l in candLambda:
    inter = InterpolatedLM(fsents, vocab, 10, 5, l)
    ce, _ = inter.crossEntropyAndPerplexity(val)
    if(ce < optimalCE):
        optimalCE = ce
        optimalL = l

print(optimalL, optimalCE)

0.5 5.581870145212111


Evaluate on the test set

In [28]:
fsents, vocab = filterFreq(fltrain, 10)

In [29]:
bgrm = BigramLM(fsents, vocab, 10)
tgrm = TrigramLM(fsents, vocab, 5)

In [30]:
bgrm.crossEntropyAndPerplexity(test)

(7.038295003468157, 131.44313670503732)

In [31]:
tgrm.crossEntropyAndPerplexity(test)

(7.334055202542064, 161.35060784129482)

Sentence completion

In [32]:
from queue import Queue

def score(sentence, model):
    logProb = 0.0
    for i in range(1, len(sentence)):
        w1, w2 = sentence[i-1], sentence[i]
        prob = model.prob(w1, w2)
        logProb += math.log(prob, 2)
    return logProb

def beamSearch(sentence, model, depth, k):
    q = Queue()
    finalResults = []
    q.put((0, sentence)) 

    while not q.empty():
        d, s = q.get()
        if(d == depth):
            finalResults.append(s)
            continue

        candidateSentences = []
        for w in model.vocab:
            if(w in {'<UNK>', '<START>', '<START1>', '<START2>'}):
                continue
            if(w == s[-1]): ## Prevent consecutive duplicates
                continue
            candidateSentences.append(s + [w])

        ## Keep top k candidates by score
        topK = sorted(candidateSentences, key=lambda x: score(x, model), reverse = True)[:k]

        ## Add next depth candidates to queue
        for cand in topK:
            q.put((d + 1, cand))

    ## Return final sentences as strings
    finalResults = sorted(finalResults, key=lambda x: score(x, model), reverse = True)[:k]
    return list(map(lambda x: ' '.join(x), finalResults))

In [33]:
prompt = ['I', 'would', 'like', 'to', 'commend', 'the']
beamSearch(prompt, bgrm, depth = 3, k = 7)

['I would like to commend the editor of the',
 'I would like to commend the most of the',
 'I would like to commend the first of the',
 'I would like to commend the editor of a',
 'I would like to commend the editor of course',
 'I would like to commend the editor of his',
 'I would like to commend the new , the']

In [34]:
prompt = ['I', 'would', 'like', 'to', 'commend', 'the']
beamSearch(prompt, bgrm, depth = 5, k = 5)

['I would like to commend the editor of the editor of',
 'I would like to commend the editor of the new york',
 'I would like to commend the editor of the editor old',
 'I would like to commend the editor of the editor i',
 'I would like to commend the editor of the editor being']

In [35]:
prompts = [s[0:5] for s in test][0:10]

In [36]:
for p in prompts:
    print(*beamSearch(p, bgrm, depth = 3, k = 5), sep = '\n')

when east germans fled to the editor of
when east germans fled to the new york
when east germans fled to have to the
when east germans fled to the editor old
when east germans fled to the editor i
it was part of a good , the
it was part of a man to the
it was part of a good , and
it was part of a man , the
it was part of a great , the
yet they supported the eisenhower man to the
yet they supported the eisenhower county in the
yet they supported the eisenhower county , the
yet they supported the eisenhower man , the
yet they supported the eisenhower old , the
the state department tacitly rejected man to the
the state department tacitly rejected county in the
the state department tacitly rejected county , the
the state department tacitly rejected man , the
the state department tacitly rejected old , the
certainly , the meaning is in the editor
certainly , the meaning is in the inquirer
certainly , the meaning is in the new
certainly , the meaning is the editor of
certainly , the meaning 